<left>
<table style="margin-top:0px; margin-left:0px;">
<tr>
  <td><img src="https://raw.githubusercontent.com/worm-portal/WORM-Figures/master/style/worm.png" alt="WORM" title="WORM" width=50/></td>
  <td><h1 style=font-size:30px>Charge balancing on a column in the input file</h1><br />
</tr>
</table>
</left>

You can add a column called `charge_balance_on` in your water chemistry input file (with `species` as the subheader). This allows you to set the charge balance constraint for each sample row.

Use the file browser to open `input_example_wrm_cb.csv` to see an example of this. The file contains data for seven different hydrothermal fluid samples.

The column `charge_balance_on` sets the charge balance constraint for each sample. The samples *Bison Pool* and *Ambergris* will adjust bicarbonate, HCO3-, to achieve charge balance. *Spray Geyser Source* will be balanced on Cl-. *Crater Hills Geyser*, *Gas Crack*, and *Snail* will be balanced on H+ (pH). Lastly, *MLS Source* will not undergo charge balance correction because its balancing constraint is left blank.

Let's speciate the samples and what we get **without** charge balance at first:

In [1]:
import aqequil
ae = aqequil.AqEquil(db="wrm")

Loading a user-supplied thermodynamic database...
data1.wrm is now set as the active thermodynamic database.
This database is meant for rapid calculations between 0 and 350 °C at water saturation pressure.


In [2]:
speciation = ae.speciate(input_filename="input_example_wrm_cb.csv",
                         exclude=["Year", "Area"],
                         charge_balance_on='none', # no charge balancing will be done initially
                        )

Using wrm to speciate Bison Pool
Using wrm to speciate Spray Geyser Source
Using wrm to speciate Crater Hills Geyser
Using wrm to speciate Ambergris
Using wrm to speciate MLS Source
Using wrm to speciate Gas Crack
Using wrm to speciate Snail
Finished!


In [3]:
speciation.lookup("charge imbalance % of total charge")

Sample,charge imbalance % of total charge
,eq/kg.H2O
Sample,
Bison Pool,-0.0071
Spray Geyser Source,0.0372
Crater Hills Geyser,1.1659
Ambergris,10.0367
MLS Source,0.6201
Gas Crack,0.3345
Snail,0.3720


From the table above, we can see that charge imbalance is fairly low in most of these samples except for Ambergris. Ambergris has about a 10% charge imbalance.

Now let's try a speciation with charge balancing. By default, the `speciate()` function looks for a column called `charge_balance_on` in the input file to perform charge balancing so we don't need to set any special parameters in the cell below:

In [4]:
speciation = ae.speciate(input_filename="input_example_wrm_cb.csv",
                         exclude=["Year", "Area"])

Using wrm to speciate Bison Pool
Using wrm to speciate Spray Geyser Source
Using wrm to speciate Crater Hills Geyser
Using wrm to speciate Ambergris
Using wrm to speciate MLS Source
Using wrm to speciate Gas Crack
Using wrm to speciate Snail
Finished!


In [5]:
speciation.lookup("charge imbalance % of total charge")

Sample,charge imbalance % of total charge
,eq/kg.H2O
Sample,
Bison Pool,0.0000
Spray Geyser Source,-0.0000
Crater Hills Geyser,-0.0000
Ambergris,0.0000
MLS Source,0.6201
Gas Crack,0.0000
Snail,0.0000


Now all samples have been charge balanced (0% charge imbalance) except for *MLS Source* because we left the charge balancing constraint blank for that sample for the purpose of this demonstration.

The samples *Bison Pool* and *Ambergris* adjusted bicarbonate, HCO3-, to achieve charge balance. Let's compare bicarbonate before and after charge balancing:

In [6]:
speciation.lookup(["HCO3-_(input)", "HCO3-_total"])

Sample,HCO3-_(input),HCO3-_total
,Molality,molality
Sample,,
Bison Pool,5.69E-03,0.005688
Spray Geyser Source,5.41E-03,0.005410
Crater Hills Geyser,1.17E-04,0.000117
Ambergris,9.83E-04,0.001758
MLS Source,0.006784681,0.006785
Gas Crack,3.33E-05,0.000033
Snail,4.16E-05,0.000042


The concentration of bicarbonate in Ambergris changed noticably to achieve charge balance. However, the concentration of bicarbonate in Bison Pool doesn't look like it changed much at all. This is because charge imbalance in Bison Pool was -0.0071%, so there was very little adjustment that had to be done to achieve charge balance.

Now let's look at *Spray Geyser Source*, which was balanced on Cl-:

In [7]:
speciation.lookup(["Cl-_(input)", "Cl-_total"])

Sample,Cl-_(input),Cl-_total
,Molality,molality
Sample,,
Bison Pool,6.25E-03,0.006250
Spray Geyser Source,5.11E-03,0.005119
Crater Hills Geyser,2.29E-02,0.022900
Ambergris,7.61E-04,0.000761
MLS Source,0.008413957,0.008414
Gas Crack,0.012643218,0.012643
Snail,0.004659126,0.004659


Again, Crater Hills Geyser only had a 1.1659% charge imbalance so very little adjustment to Cl- had to be made to achieve balance. Also note that the concentration of Cl- is about 23 milimolal, fairly concentrated relative to other solutes. Adjusting a concentrated solute just a tad might be enough to achieve charge balance.

Finally, let's look at *Crater Hills Geyser*, *Gas Crack*, and *Snail* which were balanced on H+ (pH):

In [8]:
speciation.lookup(["pH_(input)", "pH"])

Sample,pH_(input),pH
,pH,pH
Sample,,
Bison Pool,7.64,7.6400
Spray Geyser Source,8.021,8.0210
Crater Hills Geyser,3.327,3.5534
Ambergris,8.452,8.4520
MLS Source,7.85,7.8500
Gas Crack,11.67,11.6767
Snail,11.65,11.6565


It can be worth balancing on pH to achieve charge balance in relatively acidic samples (like *Crater Hills Geyser*) or alkaline samples (like *Gas Crack* and *Snail*). Since pH is a log scale, small changes represent large adjustments to concentrations of H+. Therefore, small changes in pH might be enough to achieve charge balance in very acidic and alkaline conditions.

Why is this favorable? Consider the fact that pH meters have some analytical error associated with measurement due to calibration and fluctuations. If you only need to adjust pH by a little bit during speciation to achieve charge balance, this adjustment might be within the error of the pH meter in very acidic or alkaline samples. Therefore, it might be favorable to choose pH as your balancing constraint in these kinds of samples because adjusting solutes like Cl-, HCO3-, etc. might require changing their concentration(s) beyond the analytical error associated with them.

End of tutorial.